# Make pixel-to-voltage map

In [ ]:
# Imports
import os
import numpy as np
import pickle
import time
import cv2
import nidaqmx
import matplotlib.pyplot as plt
from pypylon import pylon
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Set up tasks
Task_x = nidaqmx.Task()
Task_x.ao_channels.add_ao_voltage_chan("Dev2/ao0")
Task_y = nidaqmx.Task()
Task_y.ao_channels.add_ao_voltage_chan("Dev2/ao1")
Task_x.ao_channels.all.ao_max = 10
Task_x.ao_channels.all.ao_min = -10
Task_y.ao_channels.all.ao_max = 10
Task_y.ao_channels.all.ao_min = -10

## Manually find corners 

In [ ]:
# Adjust to find the voltages for the four corners
x= 4.8
y= 5.8
Task_x.write(x, auto_start=True)
Task_y.write(y, auto_start=True)

## Scan mirror galvanometers across stimulation plane to make pixel-to-voltage dictionary 

In [ ]:
# Add manually determined corner voltages
initial_voltage_x = 4.8
final_voltage_x = -6.2
initial_voltage_y = 5.8
final_voltage_y = -5.9

# Number of samples along each axis. 100 x 100 is suggested.
samples_x = 100 
samples_y = 100 

# Set up 2D voltage grid
array_x = np.zeros((samples_x, samples_y))
for i in range(samples_y):
    if i % 2 == 0:
        array_x[:,i] = np.linspace(initial_voltage_x,final_voltage_x,samples_x)
    else:
        array_x[:,i] = np.linspace(final_voltage_x,initial_voltage_x,samples_x)
array_x = array_x.reshape((samples_y*samples_x), order='F')
array_y = np.linspace(initial_voltage_y,final_voltage_y,samples_y).repeat(samples_x)

# Scan across grid while capturing peak pixel locations
Task_x.write(initial_voltage_x, auto_start=True)
Task_y.write(initial_voltage_y, auto_start=True)
#time.sleep(10)
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using device:", camera.GetDeviceInfo().GetModelName())
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
camera.StartGrabbing(pylon.GrabStrategy_OneByOne)
x_pixel = []
y_pixel = []
x_voltage = []
y_voltage = []
for i in range(samples_x*samples_y):
    Task_x.write(array_x[i], auto_start=True)
    Task_y.write(array_y[i], auto_start=True)
    print(array_x[i], array_y[i])
    count = 0
    mean_max_loc = []
    while count < 5:
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException) #adjust exposure time as needed
        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            img = image.GetArray()[:,:,0]          
            if (count > 0) & (count < 4): 
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img)
                mean_max_loc.append(max_loc)
            count += 1
    mean_max_loc = np.array(mean_max_loc).mean(axis=0)
    x_pixel.append(mean_max_loc[0])
    y_pixel.append(mean_max_loc[1])
    x_voltage.append(array_x[i])
    y_voltage.append(array_y[i])    
camera.StopGrabbing()

# Save x_pixel and y_pixel values
with open(r'path', 'w', newline='') as file: ### Add path
    writer = csv.writer(file)
    writer.writerow(x_pixel)
    writer.writerow(y_pixel)
    
# Fit x_pixel and y_pixel values with 2D polynomial
poly = PolynomialFeatures(degree=3)
X = np.vstack([x_pixel, y_pixel]).T
X_poly = poly.fit_transform(X)
model_x = LinearRegression().fit(X_poly, x_voltage)
model_y = LinearRegression().fit(X_poly, y_voltage)
X_all = np.vstack([x_pixel_all, y_pixel_all]).T
X_all_poly = poly.transform(X_all)
Z_xfit = model_x.predict(X_all_poly)
Z_yfit = model_y.predict(X_all_poly)
Z_xfit[Z_xfit > 10] = 10
Z_xfit[Z_xfit < -10] = -10
Z_yfit[Z_yfit > 10] = 10
Z_yfit[Z_yfit < -10] = -10

# Make dictionary
pixel_to_voltage = {}
for i in range(Z_xfit.size):
    pixel_to_voltage[(x_pixel_all[i],y_pixel_all[i])] = (Z_xfit[i], Z_yfit[i])
    
# Save dictionary
with open(os.path.join(r'path', 'dict.pickle'), 'wb') as fp: ### Add path
    pickle.dump(pixel_to_voltage, fp)
print('dict.pickle created')

## Make a voltage scanning image

In [ ]:
# Add manually determined corner voltages
initial_voltage_x = 4.5
final_voltage_x = -5.9
initial_voltage_y = 5.5
final_voltage_y = -5.6

# Number of samples along each axis. 32 x 32 is suggested.
samples_x = 32 
samples_y = 32 

# Set up 2D voltage grid
array_x = np.zeros((samples_x, samples_y))
for i in range(samples_y):
    if i % 2 == 0:
        array_x[:,i] = np.linspace(initial_voltage_x,final_voltage_x,samples_x)
    else:
        array_x[:,i] = np.linspace(final_voltage_x,initial_voltage_x,samples_x)
array_x = array_x.reshape((samples_y*samples_x), order='F')
array_y = np.linspace(initial_voltage_y,final_voltage_y,samples_y).repeat(samples_x)

# Scan across grid while capturing peak pixel locations
Task_x.write(initial_voltage_x, auto_start=True)
Task_y.write(initial_voltage_y, auto_start=True)
#time.sleep(10)
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using device:", camera.GetDeviceInfo().GetModelName())
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
camera.StartGrabbing(pylon.GrabStrategy_OneByOne)
images = np.zeros((samples_x*samples_y,1100,1100)) #adjust resolution as needed for camera cropping
for i in range(samples_x*samples_y):
    Task_x.write(array_x[i], auto_start=True)
    Task_y.write(array_y[i], auto_start=True)
    count = 0
    while count < 5:
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException) #adjust exposure time as needed
        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            img = image.GetArray()[:,:,0]
            if count == 2:
                images[i] += img   
            count += 1      
camera.StopGrabbing()

# Save the image
filename = r'path.tif' ### Add path
image = images.sum(axis=0).astype('uint8')
image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) 
cv2.imwrite(filename, image) 

## Make a pixel scanning image to test dictionary

In [ ]:
# Add same manually determined corner voltages
initial_voltage_x = 4.5
final_voltage_x = -5.9
initial_voltage_y = 5.5
final_voltage_y = -5.6

#Find equivalent initial and final locations
invert_dict = {v: k for k, v in pixel_to_voltage.items()}
locs = np.array([loc for loc in invert_dict.keys()])
mins = []
for x,y in locs:
    mins.append(np.abs(initial_voltage_x-x).min()+np.abs(initial_voltage_y-y).min())
initial_loc = locs[np.array(mins).argmin()]
initial_loc = invert_dict[initial_loc[0],initial_loc[1]]
mins = []
for x,y in locs:
    mins.append(np.abs(final_voltage_x-x).min()+np.abs(final_voltage_y-y).min())
final_loc = locs[np.array(mins).argmin()]
final_loc = invert_dict[final_loc[0],final_loc[1]]

# Number of samples along each axis. 32 x 32 is suggested.
samples_x = 32 
samples_y = 32 

# Set up 2D voltage grid using dictionary
array_y = np.zeros((samples_x, samples_y))
for i in range(samples_x):
    if i % 2 == 0:
        array_y[:,i] = np.linspace(initial_loc[1],final_loc[1],samples_y)
    else:
        array_y[:,i] = np.linspace(final_loc[1],initial_loc[1],samples_y)
array_y = array_y.reshape((samples_y*samples_x), order='F').astype('int')
array_x = np.linspace(initial_loc[0],final_loc[0],samples_x).repeat(samples_y).astype('int')
voltages = pixel_to_voltage[array_x[0],array_y[0]]

# Scan across grid while capturing peak pixel locations
Task_x.write(voltages[0], auto_start=True)
Task_y.write(voltages[1], auto_start=True)
#time.sleep(10)
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using device:", camera.GetDeviceInfo().GetModelName())
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
camera.StartGrabbing(pylon.GrabStrategy_OneByOne)
max_locs = []
images = np.zeros((samples_x*samples_y,1100,1100)) #adjust resolution as needed for camera cropping
for i in range(samples_x*samples_y):
    voltages = pixel_to_voltage[array_x[i],array_y[i]]
    Task_x.write(voltages[0], auto_start=True)
    Task_y.write(voltages[1], auto_start=True)
    count = 0
    while count < 5:
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException) #adjust exposure time as needed
        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            img = image.GetArray()[:,:,0]
            if count == 2:
                images[i] += img   
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img)
                max_locs.append(max_loc)
            count += 1
camera.StopGrabbing()

# Save image and arrays
filename = r'path\filename.tif' ### Add path
image = images.sum(axis=0).astype('uint8')
image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) 
cv2.imwrite(filename, image) 
np.save(r'path\array_y_pixel.npy',array_y) ### Add path
np.save(r'path\array_x_pixel.npy',array_x) ### Add path
np.save(r'path\pixel_max_locs.npy',np.array(max_locs)) ### Add path